In [1]:
import pandas as pd
import re

In [2]:
dataset = pd.read_json('/kaggle/input/output/output_cy_data.json')
full_audio_segments = '/kaggle/input/full-audio-segments/segments'

In [3]:
df = pd.DataFrame(dataset)

In [4]:
df.head()

,audio_filepath,start,end,text
0,./Audio_Files/_v65uDbQTvg.wav,13.559,18.000,Λοιπόν Μαρία μου καλωσόρισες Είναι
1,./Audio_Files/_v65uDbQTvg.wav,15.440,20.199,τιμή μου που είσαι η πρώτη μας καλεσμένη
2,./Audio_Files/_v65uDbQTvg.wav,18.000,23.680,στο
3,./Audio_Files/_v65uDbQTvg.wav,20.199,26.279,Ουάου έννα πω εγώ να ξέρεις είμαι
4,./Audio_Files/_v65uDbQTvg.wav,23.680,28.439,καλότυχη Φερνογούρης συνήθως είμαι


In [5]:


# Заменить префикс пути в столбце
df['audio_filepath'] = df['audio_filepath'].str.replace('./Audio_Files/', '/kaggle/input/audio-files/Audio_Files/', regex=False)

In [6]:
df

,audio_filepath,start,end,text
0,/kaggle/input/audio-files/Audio_Files/_v65uDbQ...,13.559,18.000,Λοιπόν Μαρία μου καλωσόρισες Είναι
1,/kaggle/input/audio-files/Audio_Files/_v65uDbQ...,15.440,20.199,τιμή μου που είσαι η πρώτη μας καλεσμένη
2,/kaggle/input/audio-files/Audio_Files/_v65uDbQ...,18.000,23.680,στο
3,/kaggle/input/audio-files/Audio_Files/_v65uDbQ...,20.199,26.279,Ουάου έννα πω εγώ να ξέρεις είμαι
4,/kaggle/input/audio-files/Audio_Files/_v65uDbQ...,23.680,28.439,καλότυχη Φερνογούρης συνήθως είμαι
...,...,...,...,...
27300,/kaggle/input/audio-files/Audio_Files/rJlH5ysj...,2968.119,2976.160,όι μωρέ όι να μην έρθει στη γιορτή εν
27301,/kaggle/input/audio-files/Audio_Files/rJlH5ysj...,2972.559,2976.160,έννα προλάβω καθόλου να σε
27302,/kaggle/input/audio-files/Audio_Files/rJlH5ysj...,2976.440,2982.440,δω εισιτήριο για λονδίνο μου κράτησες
27303,/kaggle/input/audio-files/Audio_Files/rJlH5ysj...,2980.079,2985.160,ναι άρκοψες πετάς μετά τα


In [68]:
import librosa
import soundfile as sf
import pandas as pd
import os
from tqdm import tqdm
import torch
import torchaudio
def create_segments_for_hf(df, output_dir="/kaggle/working/segments"):
   os.makedirs(output_dir, exist_ok=True)
   
   # Проверяем GPU
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Используется устройство: {device}")
   
   new_data = []
   skipped_files = []
   
   for idx, row in tqdm(df.iterrows(), total=len(df)):
       try:
           # Проверяем существование файла
           if not os.path.exists(row["audio_filepath"]):
               skipped_files.append(row["audio_filepath"])
               continue
           
           # Загружаем на GPU если доступно
           if torch.cuda.is_available():
               # Используем torchaudio для GPU
               audio, sr = torchaudio.load(row["audio_filepath"])
               audio = audio.to(device)
               
               # Ресэмплинг на GPU
               if sr != 24000:
                   resampler = torchaudio.transforms.Resample(sr, 24000).to(device)
                   audio = resampler(audio)
                   sr = 24000
               
               # Нарезка
               start_sample = int(row["start"] * sr)
               end_sample = int(row["end"] * sr)
               segment = audio[0, start_sample:end_sample]  # берем первый канал
               
               # Переводим обратно на CPU для сохранения
               segment = segment.cpu().numpy()
           else:
               # CPU версия
               audio, sr = librosa.load(row["audio_filepath"], sr=24000)
               start_sample = int(row["start"] * sr)
               end_sample = int(row["end"] * sr)
               segment = audio[start_sample:end_sample]
           
           # Получаем имя исходного файла без расширения
           original_filename = os.path.splitext(os.path.basename(row["audio_filepath"]))[0]
           
           # Сохранение с именем исходного файла
           filename = f"{original_filename}_segment_{idx:06d}.wav"
           output_path = os.path.join(output_dir, filename)
           sf.write(output_path, segment, sr)
           
           # Подготовка данных для HF
           new_data.append({
               'file_name': filename,
               'text': row['text'],
               'duration': row['end'] - row['start'],
               'original_file': row['audio_filepath']
           })
           
       except Exception as e:
           print(f"Ошибка {idx}: {e}")
   
   # Создаем новый датафрейм
   hf_df = pd.DataFrame(new_data)
   hf_df.to_csv(os.path.join(output_dir, "metadata.csv"), index=False)
   
   print(f"Пропущено {len(skipped_files)} отсутствующих файлов")
   if skipped_files:
       print("Отсутствующие файлы:")
       for file in set(skipped_files):
           print(f"  - {file}")
   
   return hf_df

In [69]:
df_half = df.iloc[len(df)//2:]

In [ ]:
# Проверяем GPU
print(f"CUDA доступен: {torch.cuda.is_available()}")
if torch.cuda.is_available():
   print(f"Устройство: {torch.cuda.get_device_name(0)}")
# Использование
result_df = create_segments_for_hf(df_half)
print(f"Готово! {len(result_df)} сегментов в папке /kaggle/working/segments/")

CUDA доступен: True
Устройство: Tesla T4
Используется устройство: cuda


 14%|█▍        | 1971/13653 [20:53<2:36:09,  1.25it/s]

In [75]:
import tarfile

output_filename = '/kaggle/working/full.tar.gz'
source_dir = '/kaggle/working'

with tarfile.open(output_filename, 'w:gz') as tar:
    tar.add(source_dir, arcname='.')


In [76]:
from IPython.display import FileLink
FileLink(output_filename)


/kaggle/working/full.tar.gz